In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
import os
import sys
import transformers
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig


In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Define 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normal Float 4 (more accurate)
    bnb_4bit_use_double_quant=True,  # Improves precision
    bnb_4bit_compute_dtype="bfloat16",  # Compute in fp16 for efficiency
)


tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/m2m100_418M",
    quantization_config=bnb_config,  # Use the defined quantization config
    device_map="auto"  # Automatically map layers to GPUs/CPUs
)

# Confirm model loaded successfully
print(model.hf_device_map)  # Displays layer-to-device allocation

{'': 0}


In [3]:
from datasets import load_dataset
from datasets import DatasetDict

# Load only 30% of the dataset
dataset = load_dataset("csv", data_files={"train": "wikimatrix_en_hi.csv"}, split="train[:30%]")

# Split into train and test sets (e.g., 80% train, 20% test)
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Further split the test set into validation and test (e.g., 50-50 split of the 20%)
validation_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)

# Combine splits into a DatasetDict
raw_dataset = {
    "train": train_test_split["train"],
    "validation": validation_test_split["train"],
    "test": validation_test_split["test"]
}

final_dataset = DatasetDict(raw_dataset)

# Inspect the resulting dataset
print(final_dataset)


DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 55550
    })
    validation: Dataset({
        features: ['input', 'target'],
        num_rows: 6944
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 6944
    })
})


In [6]:
raw_dataset['train'][1]


{'input': 'India Today 12012004.', 'target': 'इनकी आबादी २०७१११ है।'}

In [7]:
tokenizer("Hello, this is a sentence!")



{'input_ids': [128022, 65761, 4, 15911, 117, 8, 8931, 6226, 30, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="hi")


In [9]:
# Now tokenize
with tokenizer.as_target_tokenizer():
    print(tokenizer(["इनकी आबादी २०७१११ है।"]))

{'input_ids': [[128036, 16942, 5143, 1843, 7544, 7586, 82635, 8940, 8940, 8940, 776, 209, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


c:\Users\akthe\miniconda3\envs\WPanda\Lib\site-packages\transformers\tokenization_utils_base.py:3957: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
    # Extract inputs and targets from the dataset's features
    inputs = examples["input"]
    targets = examples["target"]

    # Tokenize the input text
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Tokenize the target text
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    # Add tokenized targets as labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [11]:
preprocess_function(raw_dataset["train"][:2])


{'input_ids': [[128022, 1658, 23404, 90093, 20341, 123838, 104702, 193, 84864, 432, 1224, 114961, 241, 11072, 118225, 28, 49183, 98216, 1197, 124163, 4680, 986, 113605, 117, 124474, 713, 1224, 1197, 75086, 9066, 7480, 5, 2], [128022, 13512, 90173, 161, 9433, 5716, 2525, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[128036, 1843, 4812, 36476, 111303, 54493, 698, 27419, 59960, 1564, 69006, 34667, 3282, 12018, 15239, 53882, 978, 84864, 776, 18363, 12605, 24016, 9034, 986, 37735, 18443, 27419, 3631, 80166, 1839, 30569, 4166, 31775, 8053, 27741, 776, 209, 2], [128036, 16942, 5143, 1843, 7544, 7586, 82635, 8940, 8940, 8940, 776, 209, 2]]}

In [12]:
# Map the preprocessing function to the DatasetDict
tokenized_datasets = final_dataset.map(preprocess_function, batched=True)

# Inspect the tokenized dataset
print(tokenized_datasets)


DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 185167
    })
    validation: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 23146
    })
    test: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 23146
    })
})


In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [14]:
import evaluate

metric = evaluate.load("sacrebleu")

In [15]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [16]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="../Models",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/46292 [00:00<?, ?it/s]